In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4258/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_4258/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_8_5,0.986135,0.766640,0.882078,0.982080,0.006811,0.138532,0.100439,0.012488,0.043716,0.082528,0.971376,0.083827,75.978482,147.733262,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_11_4,0.985051,0.766066,0.891451,0.987031,0.007343,0.138874,0.092456,0.007181,0.046034,0.085693,0.969138,0.087043,75.827931,147.582711,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_11_7,0.985167,0.764708,0.889548,0.983401,0.007286,0.139680,0.094077,0.009191,0.046108,0.085360,0.969377,0.086704,75.843506,147.598286,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_11_0,0.985195,0.763487,0.894795,0.990118,0.007273,0.140405,0.089608,0.005471,0.045677,0.085281,0.969434,0.086624,75.847204,147.601984,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_12_0,0.985407,0.762718,0.888605,0.976343,0.007169,0.140861,0.094879,0.009975,0.045778,0.084668,0.969872,0.086002,75.876050,147.630830,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,model_1_0_0,0.005567,-0.165003,0.127768,0.229011,0.488497,0.691596,0.742916,0.256409,0.434925,0.698925,-0.354121,0.709933,35.432845,72.397429,"Hidden Size=[4], regularizer=0.02, learning_ra..."
656,model_5_1_0,0.106580,-0.179246,0.454609,0.133506,0.438876,0.700052,0.464532,0.560432,0.353315,0.662477,1.705912,0.672910,291.647077,606.933231,"Hidden Size=[36], regularizer=0.02, learning_r..."
657,model_4_1_3,0.069800,-0.206502,0.376848,0.259867,0.456944,0.716232,0.530764,0.588990,0.379542,0.675976,2.804024,0.686622,195.566390,406.481956,"Hidden Size=[24], regularizer=0.02, learning_r..."
658,model_2_0_0,0.057747,-0.217116,0.272035,0.106233,0.462864,0.722533,0.620037,0.517378,0.407429,0.680341,-0.945296,0.691056,67.540643,139.295423,"Hidden Size=[8], regularizer=0.02, learning_ra..."
